## In this notebook we'll parse CSV statement from Bank of america and try to visualize spending

1. let's download a CSV statement from Bank Of America website

TBD screenshot here

### reading CSV statement into a list of plain dicts.

In [19]:
from csv import DictReader

with open('/Users/rinozemt/Downloads/February2020_4216.csv') as csvfile:
    reader = DictReader(csvfile)
    transactions_list = list(dict(x) for x in reader)

### standardize columns data, now it's all strings.

```
amount -> decimal
date -> datetime
```

In [55]:
## TODO: that should be in the yaml config file.
bofa_columns_meta = {
    "Posted Date": {
        "name": "date",
        "type": "datetime",
        "description": "date of the transaction"
    },
    "Reference Number": {
        "name": "ref_num",
        "type": "str",
        "description": "transaction / reference number"
    },
    "Payee": {
        "name": "payee",
        "type": "str",
        "description": "reciever of the money" 
    },
    "Address": {
        "name": "address",
        "type": "str",  # for now, may be address as a type???
        "description": "address where trasaction happened, most likely just a city"
    },
    "Amount": {
        "name": "amount",
        "type": "decimal",
        "description": "amount of money paid"  # wtf there is no currency
    }
}

from dateutil.parser import parse
from decimal import Decimal
from typing import List


def standardize_item(key: str, value: str):
    """Takes key, value and returns new key, value
    
    For example:
    standardize_item("Amount", "27.97") returns ("amount", Decimal("27.97"))
    """
    column_meta = bofa_columns_meta[key]
    new_key = column_meta["name"]
    column_type = column_meta["type"]
    if column_type == "datetime":
        new_value = parse(value)
    elif column_type == "decimal":
        new_value = Decimal(value)
    elif column_type == "str":
        new_value = str(value)
        new_value = re.sub('\s+',' ', new_value.lower().strip())
    else:
        raise Exception("unknown column type")
    return new_key, new_value
        

def standardize_transaction(transaction: dict):
    formatted_tx = {}
    for k, v in transaction.items():
        new_key, new_value = standardize_item(k, v)
        formatted_tx[new_key] = new_value
    return formatted_tx

def standardize_results(transactions: List[dict]):
    standard_results = []
    for tx in transactions:
        standardized_tx = standardize_transaction(tx)
        standard_results.append(standardized_tx)
    return standard_results

st_r = standardize_results(transactions_list)

In [56]:
st_r

[{'date': datetime.datetime(2020, 2, 12, 0, 0),
  'ref_num': '',
  'payee': 'interest charged on purchases',
  'address': '',
  'amount': Decimal('-27.81')},
 {'date': datetime.datetime(2020, 2, 12, 0, 0),
  'ref_num': '04383204320021200050652',
  'payee': 'payment - thank you',
  'address': '',
  'amount': Decimal('2426.09')},
 {'date': datetime.datetime(2020, 2, 11, 0, 0),
  'ref_num': '24431060041975016990209',
  'payee': 'safeway #308 half moon bayca',
  'address': 'half moon bay ca',
  'amount': Decimal('-21.41')},
 {'date': datetime.datetime(2020, 2, 10, 0, 0),
  'ref_num': '',
  'payee': 'late fee for payment due',
  'address': '',
  'amount': Decimal('-25.00')},
 {'date': datetime.datetime(2020, 2, 10, 0, 0),
  'ref_num': '24492150039637129230449',
  'payee': 'philz coffee www.philzcoffca',
  'address': 'www.philzcoff ca',
  'amount': Decimal('-3.60')},
 {'date': datetime.datetime(2020, 2, 7, 0, 0),
  'ref_num': '24492150038637025023659',
  'payee': 'instacart httpsinstacarca',

In [61]:
import re

bofa_matchers = [
    {
        "match": {
            "type": "exact",
            "exact": "late fee for payment due"
        },
        "assign": {
            "category": "fees",
            "tags": ["important", "fees"],
            "description": "Credit Card late payment fee",
            "merchant": "Bank Of America"
        }
    },
    {
        "match": {
            "type": "exact",
            "exact": "interest charged on purchases"
        },
        "assign": {
            "category": "fees",
            "tags": ["important", "fees"],
            "description": "Credit Card late payment fee",
            "merchant": "Bank Of America"
        }
    }, 
    {
        "match": {
            "type": "exact",
            "exact": "payment - thank you"
        },
        "assign": {
            "category": "fees",
            "tags": ["important"],
            "description": "Credit Card payment",
            "merchant": "Bank Of America"
        }
    },
    {
        "match": {
            "type": "regex",
            "regex": "^instacart .*?"
        },
        "assign": {
            "merchant": "Instacart",
            "category": "groceries",
            "tags": ["groceries", "essentials", "food"],
            "description": "Instacart Order"
        }
    },
    {
        "match": {
            "type": "regex",
            "regex": r"^walgreens #(?P<merchant_location>\d*)"
        },
        "assign": {
            "merchant": "walgreens",
            "category": "pharmacy",
            "tags": ["pharmacy", "essentials"],
            "description": "Walgreens purchase",
        }
    }
]


def try_match(payee, matcher):
    match = matcher["match"]
    if match["type"] == "exact":
        if payee == match["exact"]:
            return matcher["assign"]
    elif match["type"] == "regex":
        re_match = re.match(match["regex"], payee)
        if re_match:
            assign_values = matcher["assign"]
            if re_match.groupdict():
                assign_values.update(re_match.groupdict())
            return assign_values
    return None

def clean_addess_from_payee(tx):
    address = tx["address"]
    if not address:
        return
    payee = tx['payee']
    no_whitespace_address = address.replace(' ', '')
    no_whitespace_payee = tx['payee'].replace(' ', '')
    if no_whitespace_payee.endswith(no_whitespace_address):
        print("NEED CLEANUP")
        # need to remove address from payee
        # possibilities are:
        # sunnyvale CA -> SunnyvaleCA
        # mountain view ca -> mountain viewca
        # sunnyvale ca -> sunnyvale ca
        if payee.endswith(address):
            print("remove full address")
            tx['payee'] = payee.replace(address, '').strip()
        k = address.rfind(" ")
        address_without_last_space = address[:k]  + address[k+1:]
        print(address_without_last_space)
        if payee.endswith(address_without_last_space):
            print("remove partial address")
            tx['payee'] = payee.replace(address_without_last_space, '').strip()


def parse_payee(payee):
    for matcher in bofa_matchers:
        assign_values = try_match(payee, matcher)
        if assign_values:
            return assign_values

for tx in st_r:
    values_to_add = parse_payee(tx["payee"])
    clean_addess_from_payee(tx)
    tx_copy = tx.copy()
    if values_to_add:
        tx_copy.update(values_to_add)
    print(tx_copy)

{'date': datetime.datetime(2020, 2, 12, 0, 0), 'ref_num': '', 'payee': 'interest charged on purchases', 'address': '', 'amount': Decimal('-27.81'), 'category': 'fees', 'tags': ['important', 'fees'], 'description': 'Credit Card late payment fee', 'merchant': 'Bank Of America'}
{'date': datetime.datetime(2020, 2, 12, 0, 0), 'ref_num': '04383204320021200050652', 'payee': 'payment - thank you', 'address': '', 'amount': Decimal('2426.09'), 'category': 'fees', 'tags': ['important'], 'description': 'Credit Card payment', 'merchant': 'Bank Of America'}
{'date': datetime.datetime(2020, 2, 11, 0, 0), 'ref_num': '24431060041975016990209', 'payee': 'safeway #308', 'address': 'half moon bay ca', 'amount': Decimal('-21.41')}
{'date': datetime.datetime(2020, 2, 10, 0, 0), 'ref_num': '', 'payee': 'late fee for payment due', 'address': '', 'amount': Decimal('-25.00'), 'category': 'fees', 'tags': ['important', 'fees'], 'description': 'Credit Card late payment fee', 'merchant': 'Bank Of America'}
{'date'